<a href="https://colab.research.google.com/github/LuFernandez/neural-networks/blob/master/naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayes
## Redes Neuronales

© 2020 Lucero G. Fernandez



In [0]:
%load_ext autoreload
%autoreload 2
from IPython.display import clear_output
import pandas as pd
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Carga de datos


In [0]:
! wget "https://raw.githubusercontent.com/rn-2019-itba/Clase-2--Hiperparametros-y-Tecnicas-de-Validacion/master/Opcional/data/emails.csv"
clear_output()

In [0]:
dataset = pd.read_csv('emails.csv')

In [0]:
dataset.keys()

Index(['text', 'spam'], dtype='object')

In [0]:
dataset.spam[0]

1

In [0]:
dataset['spam']

0       1
1       1
2       1
3       1
4       1
       ..
5723    0
5724    0
5725    0
5726    0
5727    0
Name: spam, Length: 5728, dtype: int64

Se tienen 5728 emails de dos tipos: text y spam. En text están las palabras y en spam si es o no spam (1==spam)

In [0]:
#for i in range(len(dataset['spam'])):
#  if dataset.spam[i]==1:
 #   print(dataset.spam[i])

 #la cantidad de emails que son spam
 spam_count=np.sum(dataset.spam)

Probabilidad a priori de que sea spam:

In [0]:
print(spam_count/len(dataset)*100)

23.88268156424581


#  Preprocesamos los datos


In [0]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [0]:
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
emails_raw=dataset.text
emails_filtered=list()

for n in range(len(dataset)):
  tok = word_tokenize(emails_raw[n])  #tokenization
  lem = [lemmatizer.lemmatize(x,pos = 'v') for x in tok] #lemmatization
  stop = [x for x in lem if x not in stopwords.words('english')] #stop words
  stem = [stemmer.stem(x) for x in stop] #stemming
  alpha = [x for x in stem if x.isalpha()]  #filter non words

  emails_filtered.append(" ".join(alpha))




Guardo en disco

In [0]:
import pickle

with open('em_filt.pck', 'wb') as fp:
    pickle.dump(emails_filtered, fp)

In [0]:
with open ('em_filt.pck', 'rb') as fp:
    itemlist = pickle.load(fp)

Bag of Words


In [0]:
#Count Vectorizer

# from sklearn.feature_extraction.text import CountVectorizer

# count_vect = CountVectorizer(max_df=.8,min_df=10)
# raw_data=count_vect.fit_transform(emails_filtered)
# raw_data.toarray()


#TfidVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(max_df=0.8,min_df=10)
raw_data = tfidf_vect.fit_transform(itemlist)
vocabulary=tfidf_vect.get_feature_names()

##Entreno modelo de Bayes

creo set de datos 

In [0]:
len_train = int(raw_data.shape[0]*0.83)
X_train = raw_data[0:len_train]
y_train = dataset[0:len_train]['spam']

creo set de test

In [0]:
len_test = raw_data.shape[0]-len_train   #lo que no es train es test
X_test = raw_data[len_test:]
y_test = dataset[len_test:]['spam']

entreno modelo

In [0]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB(alpha=.01)
clf.fit(X_train.toarray(), y_train)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

Analizo la precisión del modelo

In [0]:
porc=sum(np.array(clf.predict(X_test.toarray()))==np.array(y_test))/len(y_test)*100
f'El porcentaje de emails clasificados correctamente es: {porc}%'

'El porcentaje de emails clasificados correctamente es: 99.76861590239798%'

Se probó con un alpha = 1, con el que se obtuvo una precisión de 99.32%, pero se obtuvieron mejores resultados con un smoothing menor, seteado a 0.01, lo que dio una precisión del 99.67%.
Además se probó tanto count vectorizer como tfid vectorizer siendo el último el que dio mejores resultados.
La precisión también aumentó cuando se incrementaron los datos usados para testear y se disminuyeron la cantidad para entrenar al modelo y a su vez también, al disminuir la mínima frecuencia de aparición de cada palabra del vocabulario, esto es, min_df.